In [90]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
from scipy import stats
import xarray as xr

In [91]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.7.1/'

In [92]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [93]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_SubsetC_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_SubsetC_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Begin_Time, Compl_Time, Peak_Time = [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [94]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [95]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [96]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [97]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [98]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [99]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/composites/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [100]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
Control_B_Compo_nc = Composite_File('composite_h3_CORI.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
Control_C_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')

In [101]:
# Open Storm Composite Files
RCP45_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP45_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
RCP45_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')

In [102]:
# Open Storm Composite Files
RCP85_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP85_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
RCP85_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004.nc')

In [103]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon)
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [104]:
# Combine Composite DFs
def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
    Compo_DF_A = Composite_DF(Compo_A, "A")
    Compo_DF_B = Composite_DF(Compo_B, "B")
    Compo_DF_C = Composite_DF(Compo_C, "C")
    Compo_DF = pandas.concat([Compo_DF_A, Compo_DF_B, Compo_DF_C]).reset_index()
    Compo_DF = Compo_DF.drop("index", axis=1)
    return (Compo_DF)

In [105]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [106]:
RCP45_Compo = Combine_Compo_DF(RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [107]:
RCP85_Compo = Combine_Compo_DF(RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [108]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [109]:
# Create Function to Find Year Diff
def Year_Diff_Find(New_Time):
    Years = [1900,1930,1960,2000,2031,2062,2100,2131,2162,2193]
    New_Time_Index = -728
    for i in range(len(Years)):
        if i < 3:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 1985 - Years[i]
                New_Time_Index = i
        elif i < 6:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
        else:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
    if New_Time_Index % 3 == 0:
        ABC = "A"
    elif New_Time_Index % 3 == 1:
        ABC = "B"
    elif New_Time_Index % 3 == 2:
        ABC = "C"
    return (int(Year_Diff), ABC)

In [110]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF):
    DF_Storm = Find_Storm(Data_DF, Code)
    Code_List = DF_Storm["Code"]
    Name_List = DF_Storm["Name"]
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    SLP = DF_Storm["SLP(hPa)"]
    Storm_Phase = DF_Storm["Storm Phase"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Year_Diff, ABC = Year_Diff_Find(New_Time[0])
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
# Find Possible Storms that Occur at the Same Time
        Compo_Storm = Compo_DF[(Compo_DF["ABC"] == ABC) & (Compo_DF["Time"] == Orig_Time)].reset_index()
# If No Storm Found:
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
# Storms Found:
        else:
            Dist_Min = [7428,-728]
            for c in range(len(Compo_Storm)):
                Dist = Find_Distance(Lat[i], Compo_Storm["Lat"][c], Lon[i], Compo_Storm["Lon"][c])
# Find Storm Closest to Storm Center
                if Dist < Dist_Min[0]:
# At Most 300km of Error in Location Permitted
                    if Dist < 300:
                        Dist_Min = [Dist, Compo_Storm["Orig Index"][c]]
                    else:
                        Dist_Min = [Dist, -728]
            Compo_Indexes[i] = Dist_Min[1]
    DF_Storm_Compo_Init = pandas.DataFrame({"Code": Code_List, "Name": Name_List, \
    "Compo Index": Compo_Indexes, "Time": New_Time, \
    "Lon": Lon, "Lat": Lat, "SLP": SLP, "Storm Phase": Storm_Phase})
# Remove Datapoints With Missing Compo Index
    DF_Storm_Compo = DF_Storm_Compo_Init[DF_Storm_Compo_Init["Compo Index"] >= 0].reset_index()
    DF_Storm_Compo = DF_Storm_Compo.drop("index", axis=1)
    return (DF_Storm_Compo)

In [111]:
def Windspeed_850hPa(Compo_nc, Compo_Index):
    U850 = numpy.array(Compo_nc.snap_U850[int(Compo_Index)])
    V850 = numpy.array(Compo_nc.snap_V850[int(Compo_Index)])
    Snap_850 = numpy.sqrt(U850 **2 + V850 **2)
    return (Snap_850)

In [112]:
# Find Precip Rate From Compo File
def Precip_Rate(Compo_nc, Compo_Index):
    Precip_ms = numpy.array(Compo_nc.snap_PRECT[int(Compo_Index)])
    Precip_mmhr = Precip_ms * 3600 * 1000
    return (Precip_mmhr)

In [113]:
# Find Precipitable Water From Compo File
def Precip_Water(Compo_nc, Compo_Index):
    Precipitable_Water = numpy.array(Compo_nc.snap_TMQ[int(Compo_Index)])
    return (Precipitable_Water)

In [114]:
# Find Outgoing Longwave Radiation and Cloud Top Temperature From Compo File
def Cloud_Temp(Compo_nc, Compo_Index):
    Outgoing_Longwave = numpy.array(Compo_nc.snap_FLUT[int(Compo_Index)])
    Sigma = 5.67 * 10**-8
    Cloud_Temp_K = (Outgoing_Longwave / (0.95 * Sigma)) ** 0.25
    Cloud_Temp_C = Cloud_Temp_K - 273.15
    return (Cloud_Temp_C)

In [115]:
# Find Surface Temperature From Compo File
def Temp_Surface(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_TS[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [116]:
# Find 850hPa Temperature From Compo File
def Temp_850hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T850[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [117]:
# Find 500hPa Temperature From Compo File
def Temp_500hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T500[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [118]:
# Find 200hPa Temperature From Compo File
def Temp_200hPa(Compo_nc, Compo_Index):
    Temp_K = numpy.array(Compo_nc.snap_T200[int(Compo_Index)])
    Temp_C = Temp_K - 273.15
    return (Temp_C)

In [119]:
# Find 500hPa Vertical Velocity From Compo File
def Omega_500hPa(Compo_nc, Compo_Index):
    Vert_Velo = numpy.array(Compo_nc.snap_OMEGA500[int(Compo_Index)])
    return (Vert_Velo)

In [120]:
# Find 200hPa Zonal Winds From Compo File
def U_200hPa(Compo_nc, Compo_Index):
    Zonal_Wind = numpy.array(Compo_nc.snap_U200[int(Compo_Index)])
    return (Zonal_Wind)

In [121]:
# Find 850hPa Max Windspeed and Wind Field Size at Each 6 Hourly Data Point
def Wind_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
    SLP = DF_Storm_Compo["SLP"]
#
# Create Array to Store Data
    Wind_Field_Info = numpy.zeros((6,len(Compo_Index)))
    Wind_Field_Info[0] = SLP
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Windspeed Snap From Compo_nc
        Snap_850 = Windspeed_850hPa(Compo_nc, Compo_Index[k])
# Find Maximum 850hPa Windspeed
        Windspeed_850 = numpy.max(Snap_850)
        Wind_Field_Info[1][k] = Windspeed_850
# Count Number of Data Points With Windspeed Above 13,18,25,33m/s
        Snap_Sort = numpy.sort(Snap_850.ravel())
        Count_13 = len(Snap_Sort[Snap_Sort >= 13])
        Count_18 = len(Snap_Sort[Snap_Sort >= 18])
        Count_25 = len(Snap_Sort[Snap_Sort >= 25])
        Count_33 = len(Snap_Sort[Snap_Sort >= 33])
        Wind_Field_Info[2][k] = Count_13
        Wind_Field_Info[3][k] = Count_18
        Wind_Field_Info[4][k] = Count_25
        Wind_Field_Info[5][k] = Count_33
#
# Add Wind Field Info Into DF Storm Compo
    DF_Storm_Compo["850hPa Winds"] = Wind_Field_Info[1]
    DF_Storm_Compo["13m/s"] = Wind_Field_Info[2]
    DF_Storm_Compo["18m/s"] = Wind_Field_Info[3]
    DF_Storm_Compo["25m/s"] = Wind_Field_Info[4]
    DF_Storm_Compo["33m/s"] = Wind_Field_Info[5]
    return (DF_Storm_Compo)

In [122]:
# Find Max Precip and Total Precip Over Area
def Precip_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
#
# Create Array to Store Data
    Precip_Field_Info = numpy.zeros((7,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find Precip Snap From Compo_nc
        Snap_Precip = Precip_Rate(Compo_nc, Compo_Index[k])
# Find Maximum Precip Rate
        Max_Precip = numpy.max(Snap_Precip)
        Precip_Field_Info[0][k] = Max_Precip
# Find Total Areal Precip
        Snap_Sort = numpy.sort(Snap_Precip.ravel())
        Areal_Precip_Total = numpy.sum(Snap_Sort)
        Precip_Field_Info[1][k] = Areal_Precip_Total
# Count Number of Data Points With Precip Rate Above 1, 5, 10mm/hr
        Count_1 = len(Snap_Sort[Snap_Sort >= 1])
        Count_5 = len(Snap_Sort[Snap_Sort >= 5])
        Count_10 = len(Snap_Sort[Snap_Sort >= 10])
        Precip_Field_Info[2][k] = Count_1
        Precip_Field_Info[3][k] = Count_5
        Precip_Field_Info[4][k] = Count_10
#
# Find Precipitable Water Snap From Compo_nc
        Snap_Precip_Water = Precip_Water(Compo_nc, Compo_Index[k])
# Find Maximum Precipitable Water
        Max_Precip_Water = numpy.max(Snap_Precip_Water)
        Precip_Field_Info[5][k] = Max_Precip_Water
# Find Areal Precipitable Water Total
        Precip_Water_Total = numpy.sum(Snap_Precip_Water.ravel())
        Precip_Field_Info[6][k] = Precip_Water_Total
# 
# Add Precip Field Info Into DF Storm Compo
    DF_Storm_Compo["Max Precip Rate"] = Precip_Field_Info[0]
    DF_Storm_Compo["Total Areal Precip"] = Precip_Field_Info[1]
    DF_Storm_Compo["1mm/hr"] = Precip_Field_Info[2]
    DF_Storm_Compo["5mm/hr"] = Precip_Field_Info[3]
    DF_Storm_Compo["10mm/hr"] = Precip_Field_Info[4]
    DF_Storm_Compo["Max Precip Water"] = Precip_Field_Info[5]
    DF_Storm_Compo["Total Precip Water"] = Precip_Field_Info[6]
    return (DF_Storm_Compo)

In [123]:
# Find Temperature Variables
def Temp_Vars_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
#
# Create Array to Store Data
    Temp_Field_Info = numpy.zeros((8,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find Each Variable From Compo_nc
        Snap_Temp_Cloud = Cloud_Temp(Compo_nc, Compo_Index[k])
        Snap_Temp_Sfc = Temp_Surface(Compo_nc, Compo_Index[k])
        Snap_Temp_850hPa = Temp_850hPa(Compo_nc, Compo_Index[k])
        Snap_Temp_500hPa = Temp_500hPa(Compo_nc, Compo_Index[k])
        Snap_Temp_200hPa = Temp_200hPa(Compo_nc, Compo_Index[k])
        Snap_Omega_500hPa = Omega_500hPa(Compo_nc, Compo_Index[k])
        Snap_U_200hPa = U_200hPa(Compo_nc, Compo_Index[k])
#
# Find Minimum Cloud Top Temperature
        Min_Temp_Cloud = numpy.min(Snap_Temp_Cloud)
# Find Mean Surface, 850hPa, 500hPa, 200hPa Temperatures Within 1 Lat/Lon of Storm Center
        Mean_Temp_Sfc = numpy.mean(Snap_Temp_Sfc[17:23,17:23])
        Mean_Temp_850hPa = numpy.mean(Snap_Temp_850hPa[17:23,17:23])
        Mean_Temp_500hPa = numpy.mean(Snap_Temp_500hPa[17:23,17:23])
        Mean_Temp_200hPa = numpy.mean(Snap_Temp_200hPa[17:23,17:23])
        Mean_U_200hPa = numpy.mean(Snap_U_200hPa[17:23,17:23])
# Find Maximum Rising and Sinking Vertical Velocity
        Min_Omega = numpy.min(Snap_Omega_500hPa) * -1
        Max_Omega = numpy.max(Snap_Omega_500hPa)
#
# Add To Array
        Temp_Field_Info[0][k] = Min_Temp_Cloud
        Temp_Field_Info[1][k] = Mean_Temp_Sfc
        Temp_Field_Info[2][k] = Mean_Temp_850hPa
        Temp_Field_Info[3][k] = Mean_Temp_500hPa
        Temp_Field_Info[4][k] = Mean_Temp_200hPa
        Temp_Field_Info[5][k] = Mean_U_200hPa
        Temp_Field_Info[6][k] = Min_Omega
        Temp_Field_Info[7][k] = Max_Omega
#
# Add To DF Storm Compo
    DF_Storm_Compo["Min Cloud Temp"] = Temp_Field_Info[0]
    DF_Storm_Compo["Sfc Temp"] = Temp_Field_Info[1]
    DF_Storm_Compo["850hPa Temp"] = Temp_Field_Info[2]
    DF_Storm_Compo["500hPa Temp"] = Temp_Field_Info[3]
    DF_Storm_Compo["200hPa Temp"] = Temp_Field_Info[4]
    DF_Storm_Compo["200hPa U"] = Temp_Field_Info[5]
    DF_Storm_Compo["Max Rising"] = Temp_Field_Info[6]
    DF_Storm_Compo["Max Sinking"] = Temp_Field_Info[7]
    return (DF_Storm_Compo)

In [124]:
# Create New Data DF
def DF_Data_Compo(Data_DF, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
    ABC_List = ET_DF["ABC"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        DF_Storm_Compo = Find_Composite_Data(Code_List[n], Data_DF, Compo_DF)
# Find Which Compo nc To Use
        if ABC_List[n] == "A":
            Compo_nc = Compo_nc_A
        elif ABC_List[n] == "B":
            Compo_nc = Compo_nc_B
        elif ABC_List[n] == "C":
            Compo_nc = Compo_nc_C
# Apply Functions For Finding Wind Field and Precip Field
        DF_Storm_Compo = Wind_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Precip_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Temp_Vars_Find(DF_Storm_Compo, Compo_nc)
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0:
            if DF_Storm_Compo["Storm Phase"][len(DF_Storm_Compo)-1] == "Extratropical":
# Combine DF Storm Compos
                try:
                    Data_Compo = pandas.concat([Data_Compo, DF_Storm_Compo])
                except:
                    Data_Compo = DF_Storm_Compo.copy()
    Data_Compo_Final = Data_Compo.reset_index().drop("index", axis=1)
    return (Data_Compo_Final)

In [125]:
# Create New ET DF
def DF_ET_Compo(Data_Compo, ET_DF, Compo_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        ET_Storm = Find_Storm(ET_DF, Code_List[n])
        DF_Storm_Compo = Find_Storm(Data_Compo, Code_List[n])
# Find ET Begin and ET Complete Time
        Trop_Peak_Time = ET_Storm["Trop Peak Time"][0]
        Begin_Time = ET_Storm["ET Begin Time"][0]
        Compl_Time = ET_Storm["ET Complete Time"][0]
        DF_Trop_Peak = DF_Storm_Compo[DF_Storm_Compo["Time"] == Trop_Peak_Time].reset_index()
        DF_Begin = DF_Storm_Compo[DF_Storm_Compo["Time"] == Begin_Time].reset_index()
        DF_Compl = DF_Storm_Compo[DF_Storm_Compo["Time"] == Compl_Time].reset_index()
# Only Keep Storms With Complete ET Data
        if len(DF_Storm_Compo) > 0 and len(DF_Trop_Peak) and len(DF_Begin) > 0 and len(DF_Compl) > 0:
# Combine ET Storm Compos
            ET_Storm_Compo = Find_ET_Compo(Code_List[n], ET_Storm, DF_Trop_Peak, DF_Begin, DF_Compl)
            try:
                ET_Compo = pandas.concat([ET_Compo, ET_Storm_Compo])
            except:
                ET_Compo = ET_Storm_Compo.copy()
        else:
            print (Code_List[n], len(DF_Trop_Peak), len(DF_Begin), len(DF_Compl))
    ET_Compo_Final = ET_Compo.reset_index().drop("index", axis=1)
    return (ET_Compo_Final)

In [126]:
def Find_ET_Compo(Code, ET_Storm, DF_Trop_Peak, DF_Begin, DF_Compl):
    ET_Storm_Compo = ET_Storm[["Code", "Name", "Trop Peak Time", "ET Begin Time", "ET Complete Time", \
    "Trop Peak SLP", "ET Begin SLP", "ET Complete SLP"]].copy()
    Vars = ["850hPa Winds", "13m/s", "18m/s", "25m/s", "33m/s", \
    "1mm/hr", "5mm/hr", "10mm/hr", "Max Precip Rate", "Max Precip Water", "Total Precip Water", \
    "Min Cloud Temp", "Sfc Temp", "850hPa Temp", "500hPa Temp", "200hPa Temp", "200hPa U", \
    "Max Rising", "Max Sinking"]
    for m in range(len(Vars)):
        Var = Vars[m]
        Trop_Peak_Var = str("Trop Peak " + Var)
        Begin_Var = str("ET Begin " + Var)
        Compl_Var = str("ET Complete " + Var)
        ET_Storm_Compo[Trop_Peak_Var] = DF_Trop_Peak[Var][0]
        ET_Storm_Compo[Begin_Var] = DF_Begin[Var][0]
        ET_Storm_Compo[Compl_Var] = DF_Compl[Var][0]
    return (ET_Storm_Compo)

In [127]:
Control_Data_Compo = DF_Data_Compo(Control_Data, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [128]:
Control_ET_Compo = DF_ET_Compo(Control_Data_Compo, Control_ET, Control_Compo, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [129]:
RCP45_Data_Compo = DF_Data_Compo(RCP45_Data, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [130]:
RCP45_ET_Compo = DF_ET_Compo(RCP45_Data_Compo, RCP45_ET, RCP45_Compo, \
RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [131]:
RCP85_Data_Compo = DF_Data_Compo(RCP85_Data, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [132]:
RCP85_ET_Compo = DF_ET_Compo(RCP85_Data_Compo, RCP85_ET, RCP85_Compo, \
RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [229]:
# Calculate 10%, Median, 90% Percentiles
def Percentile(Array):
    Percent_10 = round(numpy.nanpercentile(Array, 10), 1)
    Median = round(numpy.nanmedian(Array), 1)
    Percent_90 = round(numpy.nanpercentile(Array, 90), 1)
    return ([Percent_10, Median, Percent_90])

In [134]:
# Calculate Statistical Significance Using KS Test
def KS_Test(Control_Array, RCP45_Array, RCP85_Array):
    P_Val_RCP45 = round(stats.kstest(Control_Array, RCP45_Array)[1], 3)
    P_Val_RCP85 = round(stats.kstest(Control_Array, RCP85_Array)[1], 3)
    return (P_Val_RCP45, P_Val_RCP85)

In [205]:
# Create DataFrame to Store Percentiles Data
def Percentile_DF(Var, Control_Array, RCP45_Array, RCP85_Array):
    Control_Percentiles = Percentile(Control_Array)
    RCP45_Percentiles = Percentile(RCP45_Array)
    RCP85_Percentiles = Percentile(RCP85_Array)
    P_Vals = KS_Test(Control_Array, RCP45_Array, RCP85_Array)
    Control_Percentiles.append(1.000)
    RCP45_Percentiles.append(P_Vals[0])
    RCP85_Percentiles.append(P_Vals[1])
    DF = pandas.DataFrame({"Var": [Var, Var, Var, Var], "Percentile": ["10%", "Median", "90%", "P Val"], \
    "Control": Control_Percentiles, "RCP4.5": RCP45_Percentiles, "RCP8.5": RCP85_Percentiles})
    return (DF)

In [206]:
# Create DataFrame For Output
def Create_Output_DF(Control_ET, RCP45_ET, RCP85_ET, Vars):
    for i in range(len(Vars)):
        for j in range(3):
            if j == 0:
                Var = "Trop Peak " + Vars[i]
            elif j == 1:
                Var = "ET Begin " + Vars[i]
            elif j == 2:
                Var = "ET Complete " + Vars[i]
            DF = Percentile_DF(Var, Control_ET[Var], RCP45_ET[Var], RCP85_ET[Var])
            if i == 0 and j == 0:
                Output_DF = DF.copy()
            else:
                Output_DF = pandas.concat([Output_DF, DF])
    return (Output_DF)

In [230]:
Create_Output_DF(Control_ET, RCP45_ET, RCP85_ET, ["Lat"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Lat,10%,20.4,24.700,23.500
1,Trop Peak Lat,Median,29.2,32.000,31.900
2,Trop Peak Lat,90%,37.7,39.100,38.200
3,Trop Peak Lat,P Val,1.0,0.101,0.049
0,ET Begin Lat,10%,26.3,31.400,32.000
1,ET Begin Lat,Median,37.5,39.100,39.800
2,ET Begin Lat,90%,45.9,47.300,45.800
3,ET Begin Lat,P Val,1.0,0.190,0.147
0,ET Complete Lat,10%,36.0,41.800,41.200
1,ET Complete Lat,Median,46.9,47.100,46.400


In [231]:
Create_Output_DF(Control_ET, RCP45_ET, RCP85_ET, ["Lon"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Lon,10%,-79.3,-79.300,-80.000
1,Trop Peak Lon,Median,-64.9,-59.500,-61.700
2,Trop Peak Lon,90%,-44.9,-47.200,-49.100
3,Trop Peak Lon,P Val,1.0,0.217,0.547
0,ET Begin Lon,10%,-76.0,-74.100,-76.400
1,ET Begin Lon,Median,-62.4,-54.400,-56.900
2,ET Begin Lon,90%,-36.0,-36.300,-36.200
3,ET Begin Lon,P Val,1.0,0.159,0.109
0,ET Complete Lon,10%,-66.4,-63.800,-64.300
1,ET Complete Lon,Median,-46.1,-42.600,-44.600


In [232]:
Create_Output_DF(Control_ET, RCP45_ET, RCP85_ET, ["SLP"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak SLP,10%,925.0,920.200,910.300
1,Trop Peak SLP,Median,954.5,949.500,949.300
2,Trop Peak SLP,90%,984.6,987.100,984.900
3,Trop Peak SLP,P Val,1.0,0.575,0.070
0,ET Begin SLP,10%,939.9,937.600,927.000
1,ET Begin SLP,Median,977.6,975.700,972.600
2,ET Begin SLP,90%,999.7,997.700,1002.600
3,ET Begin SLP,P Val,1.0,0.591,0.558
0,ET Complete SLP,10%,971.7,972.800,970.800
1,ET Complete SLP,Median,990.9,990.600,988.400


In [233]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["850hPa Winds"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 850hPa Winds,10%,38.5,39.100,41.200
1,Trop Peak 850hPa Winds,Median,56.6,57.900,59.500
2,Trop Peak 850hPa Winds,90%,72.0,72.300,76.900
3,Trop Peak 850hPa Winds,P Val,1.0,0.762,0.347
0,ET Begin 850hPa Winds,10%,31.1,33.200,31.600
1,ET Begin 850hPa Winds,Median,48.1,51.200,50.200
2,ET Begin 850hPa Winds,90%,63.8,68.100,71.400
3,ET Begin 850hPa Winds,P Val,1.0,0.503,0.375
0,ET Complete 850hPa Winds,10%,30.2,27.200,31.500
1,ET Complete 850hPa Winds,Median,38.3,39.300,40.100


In [234]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["13m/s", "18m/s", "25m/s", "33m/s"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 13m/s,10%,473.0,472.000,513.000
1,Trop Peak 13m/s,Median,945.0,1091.500,899.000
2,Trop Peak 13m/s,90%,2060.0,1936.800,2621.600
3,Trop Peak 13m/s,P Val,1.0,0.321,0.286
0,ET Begin 13m/s,10%,586.2,938.200,787.600
1,ET Begin 13m/s,Median,1367.0,1528.000,1619.000
2,ET Begin 13m/s,90%,2372.0,2938.400,2918.600
3,ET Begin 13m/s,P Val,1.0,0.247,0.100
0,ET Complete 13m/s,10%,1142.8,1463.600,1525.200
1,ET Complete 13m/s,Median,2225.0,2198.500,2455.000


In [235]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["1mm/hr", "5mm/hr", "10mm/hr"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 1mm/hr,10%,118.0,90.700,76.400
1,Trop Peak 1mm/hr,Median,227.0,207.000,260.000
2,Trop Peak 1mm/hr,90%,489.8,491.700,512.000
3,Trop Peak 1mm/hr,P Val,1.0,0.681,0.499
0,ET Begin 1mm/hr,10%,129.0,189.200,161.600
1,ET Begin 1mm/hr,Median,290.0,297.500,279.000
2,ET Begin 1mm/hr,90%,498.2,494.600,480.800
3,ET Begin 1mm/hr,P Val,1.0,0.624,0.339
0,ET Complete 1mm/hr,10%,132.6,145.900,206.400
1,ET Complete 1mm/hr,Median,294.0,315.500,315.000


In [236]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["Max Precip Rate", "Max Precip Water", "Total Precip Water"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Max Precip Rate,10%,24.1,22.100,24.300
1,Trop Peak Max Precip Rate,Median,43.3,47.100,50.800
2,Trop Peak Max Precip Rate,90%,56.8,67.800,79.300
3,Trop Peak Max Precip Rate,P Val,1.0,0.074,0.001
0,ET Begin Max Precip Rate,10%,11.1,16.900,15.000
1,ET Begin Max Precip Rate,Median,28.3,30.500,33.900
2,ET Begin Max Precip Rate,90%,51.8,67.300,75.200
3,ET Begin Max Precip Rate,P Val,1.0,0.255,0.147
0,ET Complete Max Precip Rate,10%,7.2,7.800,11.300
1,ET Complete Max Precip Rate,Median,13.9,14.300,20.200


In [237]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["Min Cloud Temp", "Sfc Temp", "850hPa Temp", "500hPa Temp", "200hPa Temp"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Min Cloud Temp,10%,-76.5,-73.700,-74.600
1,Trop Peak Min Cloud Temp,Median,-67.6,-66.300,-65.400
2,Trop Peak Min Cloud Temp,90%,-58.6,-58.600,-56.400
3,Trop Peak Min Cloud Temp,P Val,1.0,0.155,0.292
0,ET Begin Min Cloud Temp,10%,-74.6,-73.500,-69.900
1,ET Begin Min Cloud Temp,Median,-61.2,-59.700,-57.500
2,ET Begin Min Cloud Temp,90%,-50.6,-50.600,-48.300
3,ET Begin Min Cloud Temp,P Val,1.0,0.755,0.123
0,ET Complete Min Cloud Temp,10%,-60.5,-59.700,-59.900
1,ET Complete Min Cloud Temp,Median,-52.2,-50.000,-50.000


In [238]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["200hPa U"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak 200hPa U,10%,-7.7,-2.500,-1.300
1,Trop Peak 200hPa U,Median,4.7,9.800,9.200
2,Trop Peak 200hPa U,90%,21.2,22.400,30.100
3,Trop Peak 200hPa U,P Val,1.0,0.018,0.027
0,ET Begin 200hPa U,10%,4.2,7.900,3.300
1,ET Begin 200hPa U,Median,13.4,14.400,13.900
2,ET Begin 200hPa U,90%,22.5,23.400,29.200
3,ET Begin 200hPa U,P Val,1.0,0.093,0.484
0,ET Complete 200hPa U,10%,6.1,8.700,4.700
1,ET Complete 200hPa U,Median,21.4,21.300,22.100


In [239]:
Create_Output_DF(Control_ET_Compo, RCP45_ET_Compo, RCP85_ET_Compo, ["Max Rising", "Max Sinking"])

,Var,Percentile,Control,RCP4.5,RCP8.5
0,Trop Peak Max Rising,10%,4.1,4.000,4.200
1,Trop Peak Max Rising,Median,7.3,7.100,7.100
2,Trop Peak Max Rising,90%,10.2,9.500,9.800
3,Trop Peak Max Rising,P Val,1.0,0.450,0.659
0,ET Begin Max Rising,10%,2.6,2.600,2.500
1,ET Begin Max Rising,Median,5.1,5.900,5.100
2,ET Begin Max Rising,90%,9.8,9.300,9.000
3,ET Begin Max Rising,P Val,1.0,0.588,0.982
0,ET Complete Max Rising,10%,1.7,2.000,2.500
1,ET Complete Max Rising,Median,2.9,2.900,3.500
